In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import Dataset 
from torch.utils.data.dataloader import DataLoader
from torch.optim import Adam

In [0]:
fashion_mnist = datasets.FashionMNIST('', download=True)

In [167]:
fashion_mnist.data.shape

torch.Size([60000, 28, 28])

In [168]:
flatten_data = fashion_mnist.data.view(-1,28*28)
flatten_data.shape

torch.Size([60000, 784])

In [169]:
fashion_mnist.targets.shape

torch.Size([60000])

In [170]:
flatten_data.type()

'torch.ByteTensor'

```
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py in linear(input, weight, bias)
   1368     if input.dim() == 2 and bias is not Non


e:
   1369         # fused op is marginally faster
-> 1370         ret = torch.addmm(bias, input, weight.t())
   1371     else:
   1372         output = input.matmul(weight.t())

RuntimeError: Expected object of scalar type Float but got scalar type Byte for argument #2 'mat1' in call to _th_addmm
```

In [171]:
flatten_data = flatten_data.type(torch.float32)
flatten_data.type()

'torch.FloatTensor'

In [0]:
class SimpleFFNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.lin1 = nn.Linear(784, 200)
        self.lin2 = nn.Linear(200, 50)
        self.out = nn.Linear(50,10)
    
    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return F.softmax(self.out(x))

In [0]:
learner = SimpleFFNN()

In [0]:
class Fminst_Dataset(Dataset):

    def __init__(self, X, y):
        super().__init__()
        self.X = X.cuda()
        self.y = y.cuda()

    def __len__(self):
        return self.y.numel()
    
    def __getitem__(self, i):
        return self.X[i],self.y[i] 

In [201]:
fminst_ds = Fminst_Dataset(flatten_data, fashion_mnist.targets)
bs = 256
fmnist_dl = DataLoader(
    fminst_ds,
    batch_size = bs,
    shuffle = True
)
fmnist_dl

In [0]:
def accuracy(target, predicted): return (predicted.argmax(dim=1) == target).sum().item()

In [0]:
optim = Adam(learner.parameters())


In [0]:
loss = F.nll_loss

In [205]:
learner.cuda()

SimpleFFNN(
  (lin1): Linear(in_features=784, out_features=200, bias=True)
  (lin2): Linear(in_features=200, out_features=50, bias=True)
  (out): Linear(in_features=50, out_features=10, bias=True)
)

In [206]:
# training loop
epochs = 100
for i in range(epochs):
    acc = []
    losses = []
    for X, y in fmnist_dl:
        learner.train()
        #print(X)
        y_ = learner(X)
        l = loss(y_,y)
        l.backward()
        acc.append(accuracy(y, y_))
        losses.append(l.item())
        optim.step()
        optim.zero_grad()
    acc = torch.Tensor(acc).mean() / bs
    losses = torch.Tensor(losses).mean() / bs
    print(f'loss = {losses} | accuracy = {acc}')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


loss = -0.0016576111083850265 | accuracy = 0.423138290643692
loss = -0.00179679982829839 | accuracy = 0.45880985260009766
loss = -0.0018167640082538128 | accuracy = 0.4634973406791687
loss = -0.0020197294652462006 | accuracy = 0.5157080888748169
loss = -0.002037632744759321 | accuracy = 0.5202792286872864
loss = -0.0020275842398405075 | accuracy = 0.5178191661834717
loss = -0.002047361806035042 | accuracy = 0.5227892398834229
loss = -0.0020416737534105778 | accuracy = 0.5211436152458191
loss = -0.002036231802776456 | accuracy = 0.5197805762290955
loss = -0.002050737850368023 | accuracy = 0.5235205888748169
loss = -0.0020398604683578014 | accuracy = 0.5209940075874329
loss = -0.002013552701100707 | accuracy = 0.5142120718955994
loss = -0.0020672727841883898 | accuracy = 0.5278257727622986
loss = -0.002058429876342416 | accuracy = 0.52554851770401
loss = -0.0020458176732063293 | accuracy = 0.5224733948707581
loss = -0.001995642203837633 | accuracy = 0.5095911026000977
loss = -0.001991683

KeyboardInterrupt: ignored

https://pytorch.org/docs/stable/nn.functional.html#loss-functions

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
